In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
import time
from datetime import timedelta
import PIL.Image
import math
import os
import scipy.misc
from scipy.stats import itemfreq
from random import sample
import pickle
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
import pandas as pd
from zipfile import ZipFile
from io import BytesIO
import pickle
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split


/home/ashutosh/anaconda2/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
y_train=pd.read_csv("/home/ashutosh/Downloads/final/data/labels.csv")
 
y_train=pd.get_dummies(y_train["breed"]).values


In [3]:
def DataBase_creator(archivezip, nwigth, nheight, save_name):
     
    
    
    s = (len(archivezip.namelist()[:])-1, nwigth, nheight,3)
    allImage = np.zeros(s)
    k=0
    j=0

    for i in range(1,len(archivezip.namelist()[:])):
        filename = BytesIO(archivezip.read(archivezip.namelist()[i]))  
        image = PIL.Image.open(filename)  
        image = image.resize((nwigth, nheight))
        image = np.array(image)
        image = np.clip(image/255.0, 0.0, 1.0)  

        allImage[i-1]=image
        k+=1
     
    pickle.dump(allImage, open( save_name + '.p', "wb" ) )
    
     
     
     

In [4]:
archive_train = ZipFile("/home/ashutosh/Downloads/final/data/train.zip", 'r')
image_resize = 35
DataBase_creator(archivezip = archive_train, nwigth = image_resize, nheight = image_resize , save_name = "x_train" )
x_train = pickle.load( open( "x_train.p", "rb" ) )
x_train.shape

(10222, 35, 35, 3)

In [5]:
x_train, x_test, y_train, y_test = train_test_split(x_train,y_train, test_size=0.2, random_state=4)

In [6]:
def batch(x,y,start,batch_size):
    x_batch=x[start:start+batch_size]
    y_batch=y[start:start+batch_size]
    start=start+batch_size
    return x_batch,y_batch,start

In [7]:
def weight_variable(shape):
    # break simmetry
    w = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(w)

def bias_variable(shape):
    # avoid dead neurons
    b = tf.constant(0.1, shape=shape)
    return tf.Variable(b)

# pool
def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],
                          strides=[1, 2, 2, 1], padding='SAME')

# creates default conv layer
def new_conv_layer(input, num_input_channels, filter_size, num_filters, use_pooling=True):
 
    shape = [filter_size, filter_size, num_input_channels, num_filters]
 
    weights = weight_variable(shape)
    biases = bias_variable([num_filters])
 
    
    layer = tf.nn.relu(tf.nn.conv2d(input=input,
                                    filter=weights,
                                    strides=[1, 1, 1, 1],
                                    padding='SAME') + biases)

    if use_pooling: 
        return max_pool_2x2(layer), weights

    return layer, weights

def flatten_layer(layer):
    layer_shape = layer.get_shape()
    num_features = layer_shape[1:4].num_elements()

    layer_flat = tf.reshape(layer, [-1, num_features])
    return layer_flat, num_features

def new_fc_layer(input, num_inputs, num_outputs, use_relu=True): 
    weights = weight_variable([num_inputs, num_outputs])
    biases = bias_variable([num_outputs])
 
    layer = tf.matmul(input, weights) + biases
    if use_relu:
        layer = tf.nn.relu(layer)
 
    return layer

# our network!!!

# input data
x = tf.placeholder(tf.float32, shape=[None, 35,35,3], name='input_data')
x_image = tf.reshape(x, [-1,35,35,3])
# correct labels
y_ = tf.placeholder(tf.float32, shape=[None, 120], name='correct_labels')

# fist conv layer
convlayer1, w1 = new_conv_layer(x_image, 3, 5, 32)
# second conv layer
convlayer2, w2 = new_conv_layer(convlayer1, 32, 5, 64)
#3rd conv
convlayer3, w3 = new_conv_layer(convlayer2, 64, 5, 30)
#4th conv
convlayer4, w4 = new_conv_layer(convlayer3,30, 5, 64)
# flat layer
flat_layer, num_features = flatten_layer(convlayer4)
# fully connected layer
fclayer = new_fc_layer(flat_layer, num_features, 1024)
# DROPOUT
keep_prob = tf.placeholder(tf.float32)
drop_layer = tf.nn.dropout(fclayer, keep_prob)
# final layer
W_f = weight_variable([1024, 120])
b_f = bias_variable([120])
y_f = tf.matmul(drop_layer, W_f) + b_f
y_f_softmax = tf.nn.softmax(y_f)

# loss
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y_f))

# train step
train_step = tf.train.AdamOptimizer(1e-4).minimize(loss)

# accuracy
correct_prediction = tf.equal(tf.argmax(y_f_softmax, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

# init
init = tf.global_variables_initializer()

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See @{tf.nn.softmax_cross_entropy_with_logits_v2}.



# Model1(4_conv_layer)

In [8]:
num_steps =34
batch_size =240
test_size =8
saver = tf.train.Saver()
with tf.Session() as sess:

    sess.run(init)
    #print sess.run(W_f)
    for epoch in range(30):
        start=0
        for step in range(num_steps):
            x_batch,y_batch,start= batch(x_train,y_train,start,batch_size)
         
        
            if step % 10 == 0:
                train_accuracy = accuracy.eval(feed_dict={
                x:x_batch, y_: y_batch, keep_prob: 1.0})
                
            #save_path = saver.save(sess, "/home/ashutosh/Documents/tens/model2.ckpt",global_step=step)
            train_step.run(feed_dict={x: x_batch, y_: y_batch, keep_prob: 0.5})
            #print sess.run(W_f)
        print('Epoch %d,training accuracy %f' %(epoch,train_accuracy))
        
    
    
    test_accuracy = 0.0
    start=0
    for i in xrange(test_size):
          x_batch,y_batch,start= batch(x_test,y_test,start,batch_size)
          acc = accuracy.eval(feed_dict={x: x_batch, y_:y_batch, keep_prob: 1.0})
          test_accuracy += acc
    print 'avg test accuracy in %:',100*test_accuracy/(test_size)
    file_writer = tf.summary.FileWriter('/home/ashutosh/Documents/tens', sess.graph)
    file_writer.add_graph(sess.graph)

Epoch 0,training accuracy 0.008333
Epoch 1,training accuracy 0.004167
Epoch 2,training accuracy 0.004167
Epoch 3,training accuracy 0.008333
Epoch 4,training accuracy 0.012500
Epoch 5,training accuracy 0.012500
Epoch 6,training accuracy 0.008333
Epoch 7,training accuracy 0.012500
Epoch 8,training accuracy 0.012500
Epoch 9,training accuracy 0.012500
Epoch 10,training accuracy 0.008333
Epoch 11,training accuracy 0.012500
Epoch 12,training accuracy 0.008333
Epoch 13,training accuracy 0.008333
Epoch 14,training accuracy 0.020833
Epoch 15,training accuracy 0.020833
Epoch 16,training accuracy 0.020833
Epoch 17,training accuracy 0.025000
Epoch 18,training accuracy 0.025000
Epoch 19,training accuracy 0.029167
Epoch 20,training accuracy 0.029167
Epoch 21,training accuracy 0.033333
Epoch 22,training accuracy 0.045833
Epoch 23,training accuracy 0.033333
Epoch 24,training accuracy 0.025000
Epoch 25,training accuracy 0.033333
Epoch 26,training accuracy 0.016667
Epoch 27,training accuracy 0.041667
Ep

In [11]:
def weight_variable(shape):
    # break simmetry
    w = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(w)

def bias_variable(shape):
    # avoid dead neurons
    b = tf.constant(0.1, shape=shape)
    return tf.Variable(b)

# pool
def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],
                          strides=[1, 2, 2, 1], padding='SAME')

# creates default conv layer
def new_conv_layer(input, num_input_channels, filter_size, num_filters, use_pooling=True):
 
    shape = [filter_size, filter_size, num_input_channels, num_filters]
 
    weights = weight_variable(shape)
    biases = bias_variable([num_filters])
 
    
    layer = tf.nn.relu(tf.nn.conv2d(input=input,
                                    filter=weights,
                                    strides=[1, 1, 1, 1],
                                    padding='SAME') + biases)

    if use_pooling: 
        return max_pool_2x2(layer), weights

    return layer, weights

def flatten_layer(layer):
    layer_shape = layer.get_shape()
    num_features = layer_shape[1:4].num_elements()

    layer_flat = tf.reshape(layer, [-1, num_features])
    return layer_flat, num_features

def new_fc_layer(input, num_inputs, num_outputs, use_relu=True): 
    weights = weight_variable([num_inputs, num_outputs])
    biases = bias_variable([num_outputs])
 
    layer = tf.matmul(input, weights) + biases
    if use_relu:
        layer = tf.nn.relu(layer)
 
    return layer

# our network!!!

# input data
x = tf.placeholder(tf.float32, shape=[None, 35,35,3], name='input_data')
x_image = tf.reshape(x, [-1,35,35,3])
# correct labels
y_ = tf.placeholder(tf.float32, shape=[None, 120], name='correct_labels')

# fist conv layer
convlayer1, w1 = new_conv_layer(x_image, 3, 5, 32)
# second conv layer
convlayer2, w2 = new_conv_layer(convlayer1, 32, 5, 64)
#3rd conv
convlayer3, w3 = new_conv_layer(convlayer2, 64, 5, 30)
#4th conv
convlayer4, w4 = new_conv_layer(convlayer3,30, 10, 64)
#5th conv
convlayer5, w5 = new_conv_layer(convlayer4,64, 2, 10)
#6th conv
convlayer6, w6 = new_conv_layer(convlayer5,10, 5, 30)
 
# flat layer
flat_layer, num_features = flatten_layer(convlayer4)
# fully connected layer
fclayer = new_fc_layer(flat_layer, num_features, 1024)
# DROPOUT
keep_prob = tf.placeholder(tf.float32)
drop_layer = tf.nn.dropout(fclayer, keep_prob)
# final layer
W_f = weight_variable([1024, 120])
b_f = bias_variable([120])
y_f = tf.matmul(drop_layer, W_f) + b_f
y_f_softmax = tf.nn.softmax(y_f)

# loss
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y_f))

# train step
train_step = tf.train.AdamOptimizer(1e-4).minimize(loss)

# accuracy
correct_prediction = tf.equal(tf.argmax(y_f_softmax, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

# init
init = tf.global_variables_initializer()

# Model2(6_conv_layer)

In [12]:
num_steps =34
batch_size =240
test_size =8
saver = tf.train.Saver()
with tf.Session() as sess:

    sess.run(init)
    #print sess.run(W_f)
    for epoch in range(100):
        start=0
        for step in range(num_steps):
            x_batch,y_batch,start= batch(x_train,y_train,start,batch_size)
         
        
            if step % 10 == 0:
                train_accuracy = accuracy.eval(feed_dict={
                x:x_batch, y_: y_batch, keep_prob: 1.0})
                
            #save_path = saver.save(sess, "/home/ashutosh/Documents/tens/model2.ckpt",global_step=step)
            train_step.run(feed_dict={x: x_batch, y_: y_batch, keep_prob: 0.5})
            #print sess.run(W_f)
        print('Epoch %d,training accuracy %f' %(epoch,train_accuracy))
        
    
    
    test_accuracy = 0.0
    start=0
    for i in xrange(test_size):
          x_batch,y_batch,start= batch(x_test,y_test,start,batch_size)
          acc = accuracy.eval(feed_dict={x: x_batch, y_:y_batch, keep_prob: 1.0})
          test_accuracy += acc
    print "Applied 6 convolutional layer and increased number of epochs"
    print 'avg test accuracy in %:',100*test_accuracy/(test_size)
    file_writer = tf.summary.FileWriter('/home/ashutosh/Documents/tens', sess.graph)
    file_writer.add_graph(sess.graph)

Epoch 0,training accuracy 0.008333
Epoch 1,training accuracy 0.004167
Epoch 2,training accuracy 0.004167
Epoch 3,training accuracy 0.000000
Epoch 4,training accuracy 0.004167
Epoch 5,training accuracy 0.012500
Epoch 6,training accuracy 0.004167
Epoch 7,training accuracy 0.008333
Epoch 8,training accuracy 0.025000
Epoch 9,training accuracy 0.025000
Epoch 10,training accuracy 0.029167
Epoch 11,training accuracy 0.020833
Epoch 12,training accuracy 0.025000
Epoch 13,training accuracy 0.016667
Epoch 14,training accuracy 0.029167
Epoch 15,training accuracy 0.029167
Epoch 16,training accuracy 0.033333
Epoch 17,training accuracy 0.045833
Epoch 18,training accuracy 0.058333
Epoch 19,training accuracy 0.058333
Epoch 20,training accuracy 0.062500
Epoch 21,training accuracy 0.070833
Epoch 22,training accuracy 0.083333
Epoch 23,training accuracy 0.083333
Epoch 24,training accuracy 0.087500
Epoch 25,training accuracy 0.095833
Epoch 26,training accuracy 0.100000
Epoch 27,training accuracy 0.104167
Ep